In [0]:
from pyspark.sql.functions import col, count, current_timestamp

DATA_ALVO = dbutils.widgets.get("DATA_ALVO")

df_silver = spark.table("datamasters.s_inep.d_foco_queim_format_on") \
    .filter(col("datahora").like(f"{DATA_ALVO[:4]}-{DATA_ALVO[4:6]}-{DATA_ALVO[6:]}%"))

df_gold = (
    df_silver.groupBy("estado", "bioma")
    .agg(count("*").alias("qtd_focos"))
    .withColumn("data_referencia", col("datahora").substr(1, 10))
    .withColumn("processado_em", current_timestamp())
)

df_gold.write.format("delta").mode("overwrite").saveAsTable(
    "datamasters.g_inep.agg_focos_dia_estado_bioma"
)